<a href="https://colab.research.google.com/github/vineet96/vertex-ai-class/blob/main/AutoML_training_video_action_recognition_model_for_batch_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    google-cloud-storage \
                                    jsonlines \


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
PROJECT_ID = "vertexai-dev-398204"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [3]:
REGION = "us-central1"  # @param {type: "string"}

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
BUCKET_NAME = "action-recognition-golf-swing1"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://action-recognition-golf-swing1/...


In [15]:
import jsonlines
import json
from google.cloud import aiplatform, storage
from google.cloud.aiplatform import jobs

In [16]:
aiplatform.init(project=PROJECT_ID, location=REGION)

Access Model with model-id

aiplatform.Model(model_id)

In [17]:
model = aiplatform.Model("536990483889520640")

In [18]:
IMPORT_FILES = [
    "gs://automl-video-demo-data/hmdb_golf_swing_train.csv",
    "gs://automl-video-demo-data/hmdb_golf_swing_test.csv",
]

Send a batch prediction request
Send a batch prediction request to your registered model.

Get test item(s)
Now send a batch prediction request to your Vertex AI model. You use arbitrary examples out of the dataset as a test items. Don't be concerned that the examples were likely used in training the model as we just want to demonstrate how to make a prediction.



In [19]:
import_file = IMPORT_FILES[0]
test_items = ! gsutil cat $import_file | head -n2

cols = str(test_items[0]).split(",")
test_item_1 = str(cols[0])
test_label_1 = str(cols[-1])

cols = str(test_items[1]).split(",")
test_item_2 = str(cols[0])
test_label_2 = str(cols[-1])

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

gs://automl-video-demo-data/hmdb51/9_Iron_From_160_Yards_golf_f_cm_np1_ri_goo_0.avi 1.6
gs://automl-video-demo-data/hmdb51/Adam_Scott_golf_f_cm_np1_le_goo_0.avi 1.4


Make a batch input file
Now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each video. The dictionary contains the key/value pairs:

content: The Cloud Storage path to the video.
mimeType: The content type. In our example, it is a avi file.
timeSegmentStart: The start timestamp in the video to do prediction on. Note, the timestamp must be specified as a string and followed by s (second), m (minute) or h (hour).
timeSegmentEnd: The end timestamp in the video to do prediction on.


In [20]:
test_filename = "test.jsonl"
gcs_input_uri = BUCKET_URI + "/" + test_filename

# Configure the test-data
data_1 = {
    "content": test_item_1,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}
data_2 = {
    "content": test_item_2,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

# Upload the test-data to Cloud storage bucket
bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI.replace("gs://", ""))
blob = bucket.blob(blob_name=test_filename)
data = json.dumps(data_1) + "\n" + json.dumps(data_2) + "\n"
blob.upload_from_string(data)

# Check the Cloud storage uri
print(gcs_input_uri)
! gsutil cat $gcs_input_uri

gs://action-recognition-golf-swing1/test.jsonl
{"content": "gs://automl-video-demo-data/hmdb51/9_Iron_From_160_Yards_golf_f_cm_np1_ri_goo_0.avi", "mimeType": "video/avi", "timeSegmentStart": "0.0s", "timeSegmentEnd": "5.0s"}
{"content": "gs://automl-video-demo-data/hmdb51/Adam_Scott_golf_f_cm_np1_le_goo_0.avi", "mimeType": "video/avi", "timeSegmentStart": "0.0s", "timeSegmentEnd": "5.0s"}


Make the batch prediction request
Now that your Vertex AI Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

job_display_name: The human readable name for the batch prediction job.
gcs_source: A list of one or more batch request input files.
gcs_destination_prefix: The Cloud Storage location for storing the batch prediction resuls.
sync: If set to True, the call will block while waiting for the asynchronous batch job to complete.


In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="golf",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

Cleaning up
To clean up all Google Cloud resources used in this project, you can delete the Google Cloud project you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

Dataset
Model
AutoML Training Job
Batch Job
Cloud Storage Bucket

In [ ]:

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

# Delete Cloud Storage objects
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI